In [60]:
import nltk
from nltk import classify
from nltk.sentiment.vader import SentimentIntensityAnalyzer as SIA
import re, numpy, textwrap
import pandas as pd
nltk.download('vader_lexicon')

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\scpalmer\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


True

In [61]:
results = []
texts = []

In [62]:
with open('User_reviews.txt','r',encoding ='utf8') as rf:
    User_reviews = rf.read()
    
User_ratings_df = pd.read_csv('User_ratings_dataframe.txt')   
User_ratings_df = User_ratings_df.drop(columns = 'Unnamed: 0')

In [63]:
All_user_reviews_for_movie = re.split(r"'(tt\d{7})':", User_reviews)[1:]

for i in range(0, len(All_user_reviews_for_movie), 2):
    Ind_user_reviews = re.split(r"\W\W(\d+):\s\W",All_user_reviews_for_movie[i+1])[1:]
    if Ind_user_reviews != []:
        for j in range(0,len(Ind_user_reviews),2):
            review_contents = Ind_user_reviews[j+1]
            texts.append(textwrap.fill(review_contents))

In [64]:
for i in range(len(texts)):
    pol_score = SIA().polarity_scores(texts[i]) 
    pol_score['headline'] = User_ratings_df.at[i,'Id']
    results.append(pol_score)

In [65]:
pos_scores = []
neu_scores = []
neg_scores = []
com_scores = []

for i in range(len(results)):
    pos_scores.append(results[i]['pos'])
    neu_scores.append(results[i]['neu'])
    neg_scores.append(results[i]['neg'])
    com_scores.append(results[i]['compound'])

In [66]:
User_ratings_df.insert(4,'Negativity_Score',neg_scores)
User_ratings_df.insert(5,'Neutrality_Score',neu_scores)
User_ratings_df.insert(6,'Positivity_Score',pos_scores)
User_ratings_df.insert(7,'Compound_Score',com_scores)

In [67]:
errors = []
ratings_eval = []
for i in range(len(User_ratings_df.index)):
    if pd.isnull(User_ratings_df.at[i,'Rating']) is False:
        calculated_rating = User_ratings_df.at[i,'Compound_Score'] * .5 + .5
        actual_rating = eval(User_ratings_df.at[i,'Rating']) 
        ratings_eval.append(actual_rating)
        error = ((calculated_rating - actual_rating ) / actual_rating ) *100
        errors.append(error)
    if pd.isnull(User_ratings_df.at[i,'Rating']) is True:
        errors.append(numpy.nan)
        ratings_eval.append(numpy.nan)
        
User_ratings_df.insert(8,'Error',errors)
User_ratings_df.insert(9,'Evaluated_User_Rating',ratings_eval)
User_ratings_df.to_csv('User_ratings_df_with_SA_scores.txt')

In [68]:
Score_sample = User_ratings_df.sample(800,axis='index',random_state=1)
Score_sample.to_csv('Sample_user_ratings_df_with_SA_scores.txt')

In [69]:
import plotly.express as px

In [70]:
counts = []
counts.append(Score_sample.loc[Score_sample.Evaluated_User_Rating==0.1,'Evaluated_User_Rating'].count())
counts.append(Score_sample.loc[Score_sample.Evaluated_User_Rating==0.2,'Evaluated_User_Rating'].count())
counts.append(Score_sample.loc[Score_sample.Evaluated_User_Rating==0.3,'Evaluated_User_Rating'].count())
counts.append(Score_sample.loc[Score_sample.Evaluated_User_Rating==0.4,'Evaluated_User_Rating'].count())
counts.append(Score_sample.loc[Score_sample.Evaluated_User_Rating==0.5,'Evaluated_User_Rating'].count())
counts.append(Score_sample.loc[Score_sample.Evaluated_User_Rating==0.6,'Evaluated_User_Rating'].count())
counts.append(Score_sample.loc[Score_sample.Evaluated_User_Rating==0.7,'Evaluated_User_Rating'].count())
counts.append(Score_sample.loc[Score_sample.Evaluated_User_Rating==0.8,'Evaluated_User_Rating'].count())
counts.append(Score_sample.loc[Score_sample.Evaluated_User_Rating==0.9,'Evaluated_User_Rating'].count())
counts.append(Score_sample.loc[Score_sample.Evaluated_User_Rating==1.0,'Evaluated_User_Rating'].count())

In [71]:
Bar_df = pd.DataFrame(data=
                      [(0.1,counts[0]),(0.2,counts[1]),(0.3,counts[2]),(0.4,counts[3]),(0.5,counts[4]),(0.6,counts[5]),
                       (0.7,counts[6]),(0.8,counts[7]),(0.9,counts[8]),(1.0,counts[9])
                      ],columns=['Evaluated_User_Rating','Count'])

In [72]:
fig = px.bar(Bar_df, x='Evaluated_User_Rating',y='Count',title='User Rating Counts of Sample')
fig.update_xaxes(showline=True, linecolor='black',showticklabels=True,nticks=11,ticks='outside',title_text='User Rating')
fig.update_yaxes(showline=True, linecolor='black',nticks=12,ticks='outside')
fig.show()

In [73]:
import chart_studio.tools as tls
username = 'StephCPalmer' # your username
api_key = 'M8qEVu1mK0DQ8urtCkuk' # your api key - go to profile > settings > regenerate key
tls.set_credentials_file(username=username, api_key=api_key)

In [74]:
import chart_studio.plotly as py
py.plot(fig, filename = 'User_Rating_Counts_of_Sample', auto_open=True)

'https://plotly.com/~StephCPalmer/9/'

In [75]:
dates_formatted = []
for i in Score_sample.index:
    date_form = pd.to_datetime(Score_sample.at[i,'Date Posted']).date().strftime('%Y/%m/%d')
    dates_formatted.append(date_form)
Score_sample.insert(10,'Date Formatted',dates_formatted)
#https://www.programiz.com/python-programming/datetime/strftime

In [76]:
Dist_df = Score_sample.sort_values(by=['Date Formatted'])

In [77]:
fig2 = px.scatter(Dist_df,x='Date Formatted',y='Compound_Score',hover_name="Id",title='Distribution of Sample Compound SA Scores')
fig2.update_xaxes(showticklabels=False,visible = False)
fig2.update_yaxes(showticklabels=True,nticks=12,ticks='inside',
                  showline=True, linecolor='black',
                  showgrid=True,title_text= 'Compound SA Score',
                  zeroline=True, zerolinecolor='black', zerolinewidth=.1)
fig2.show()
py.plot(fig2, filename = 'Distribution of Sample Compound SA Scores', auto_open=True)

'https://plotly.com/~StephCPalmer/11/'

In [78]:
fig3 = px.box(Score_sample,x='Evaluated_User_Rating',y='Compound_Score',hover_name="Id",
              title='Distribution of Compound Sentiment Analysis Score for User Ratings')
fig3.update_xaxes(showticklabels=True,nticks=11,title_text='User Rating')
fig3.update_yaxes(showticklabels=True,nticks=12,ticks='inside',
                  showline=True, linecolor='black',
                  showgrid=True,title_text= 'Compound SA Score')
fig3.show()
py.plot(fig3, filename = 'Box dist of SA score per User ratings', auto_open=True)

'https://plotly.com/~StephCPalmer/15/'

In [79]:
Score_sample.loc[Score_sample['Id']=='tt0461804_5']

,Id,User,Rating,Date Posted,Negativity_Score,Neutrality_Score,Positivity_Score,Compound_Score,Error,Evaluated_User_Rating,Date Formatted
1998,tt0461804_5,Otis60,10/10,26 March 2012,0.109,0.86,0.031,-0.9676,-98.38,1.0,2012/03/26


In [80]:
texts[1998]

'I\'ve come to believe this movie is actually genius. I have "\n\'not seen a more effective story that tells of the \'\n\'advancement of "civilizatiion" and what that means to \'\n"mankind.it\'s summed up in the souvenir photo sequence of "\n"the two families. But it\'s the experience of the boys that "\n\'brings us to realize how sterile our world has become. Like \'\n"the negative reviewer in this chain basically states: it\'s "\n\'boring cuz nothing blows up. In our society it has become \'\n\'difficult to appreciate what life can offer--what life in \'\n\'nature, with nature, can offer. Instead, many look to \'\n"movies to escape \'what is\'. This film shows us a \'what is\' "\n\'that transcends escapism because there is a new wonder and \'\n\'mystery around every bend. But not the wonder and mystery \'\n\'of another world: the wonder of this world. And that is how \'\n\'we were meant to live. Instead, the movie holds a mirror up \'\n\'to what has happened. Why should a Steppes fat

In [52]:
import pprint
pp = pprint.PrettyPrinter()

In [58]:
pp.pprint(texts[1998])

('I\'ve come to believe this movie is actually genius. I have "\n'
 "'not seen a more effective story that tells of the '\n"
 '\'advancement of "civilizatiion" and what that means to \'\n'
 '"mankind.it\'s summed up in the souvenir photo sequence of "\n'
 '"the two families. But it\'s the experience of the boys that "\n'
 "'brings us to realize how sterile our world has become. Like '\n"
 '"the negative reviewer in this chain basically states: it\'s "\n'
 "'boring cuz nothing blows up. In our society it has become '\n"
 "'difficult to appreciate what life can offer--what life in '\n"
 "'nature, with nature, can offer. Instead, many look to '\n"
 '"movies to escape \'what is\'. This film shows us a \'what is\' "\n'
 "'that transcends escapism because there is a new wonder and '\n"
 "'mystery around every bend. But not the wonder and mystery '\n"
 "'of another world: the wonder of this world. And that is how '\n"
 "'we were meant to live. Instead, the movie holds a mirror up '\n"
 "'to w